# Backpropagation algorithm

This notebook is made for learning purposes. *Tell what is done and achieved.*

Much inspiration is taken from http://neuralnetworksanddeeplearning.com/chap2.html

## Notation for multilayer perceptrons 
We denote the unit values of the l'th layer of a neural network in vectorform as $\mathbf{a}^{(l)}$. It is given by 

$$\mathbf{a}^{(l)} = \sigma(\mathbf{z}^{(l)})$$

Where $\sigma$ is some activation function and $\mathbf{z}^{(l)}$ is given by 

$$\mathbf{z}^{(l)} = W^{(l)}\mathbf{h}^{(l-1)} + \mathbf{b}^{(l)}$$

Where $W^{(l)}$ and $\mathbf{b}^{(l)}$ are the weight matrix and bias terms, respectively, corresponding to the layer below. 


## The MNIST dataset

We import the mnist dataset, which has 60000 images each of 28x28 pixels. 

In [1]:
from tensorflow.keras.datasets import mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train = x_train.reshape(60000, 28*28)
x_train.shape

/home/morten/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


(60000, 784)

## Feedforward

We pick a batch size 

$$ X = \begin{bmatrix} \mathbf{x}_1 \\ ... \\ \mathbf{x}_n \end{bmatrix}$$

In [2]:
import numpy as np

def create_weights_and_biases(input_shape, nodes):
    """Returns two lists. A list of all weight matrices and one
    of all biases. The inputs are 
    
        input_shape: a tuple (num_observations, num_features)
        
        nodes: is a list containing the number of 
            nodes in each layer."""
    weights = []
    biases = []
    for l in range(len(nodes)):
        if l == 0:
            w = np.random.normal(size=(nodes[l], input_shape[1]))
            b = np.random.normal(size=(nodes[l]))
        else:
            w = np.random.normal(size=(nodes[l], nodes[l-1]))
            b = np.random.normal(size=(nodes[l]))
        weights.append(w)
        biases.append(b)
    
    return weights, biases

We create a neural network with 3 layers. The first layer has 100 units, the second has 100 units and the last one has 10 units. Now, the first layer with 100 units is created by applying a weight matrix to the x_train matrix. The x_train matrix has 60000 rows and 784 columns. 

Let us now denote x_train by $X$. We then wish to find a weight matrix $W^{(1)}$ and a bias vector $\mathbf{b}^{(1)}$ such that 

$$\mathbf{h}^{(1)} = XW^{(1)} + \mathbf{b}^{(1)} $$

The weight matrix $W^{(1)}$ then needs to have dimensions (784, 100) in order to fulfill the criteria while $\mathbf{b}^{(1)}$ has dimensions (60000,100). 

In [3]:
node_list = [100, 100, 10]

weights, biases = create_weights_and_biases(x_train.shape, node_list)

In [4]:
def feedforward(x, weights, biases, activations):
    """Runs through the network.
    Outputs the z's and a's."""
    z_list = []
    a_list = []
    a = x
    for w, b, activation in zip(weights, biases, activations):
        z = np.matmul(a, w.T) + b
        a = activation(z)
        
        z_list.append(z)
        a_list.append(a)
    return z_list, a_list

We now define the activation functions. We have the sigmoid and the softmax function.

$$sigmoid(x) = \frac{1}{1 + e^{-x}} $$

$$softmax(\mathbf{x})_j = \frac{e^{x_j}}{\sum_k e^{x_k}} $$

In [24]:
def sigmoid(x, deriv=False):
    if deriv == False:
        return 1 / (1 + np.exp(-x))
    else:
        return sigmoid(x) * (1 - sigmoid(x))



def softmax(x):
    # x is a row i this case.
    softmatrix = np.zeros(x.shape)
    for i, row in enumerate(x):
        exps = np.exp(row)
        softmatrix[i] = np.exp(row) / exps.sum()
    return softmatrix

## Testing the feedforward loop

In [8]:
X = x_train[:5]

z_list, a_list = feedforward(X, weights, biases, [sigmoid, sigmoid, softmax])

In [19]:
# Find the prediction of the first digit. 
a_list[-1][0].sum()
a_list[-1][0].argmax()

1.0000000000000002

In [13]:
def predictions(y_preds):
    """Calculates the predictions where the input
    is the list of a's."""
    preds=[]
    for probs in y_preds[-1]:
        pred = probs.argmax()
        preds.append(pred)
    return preds

In [14]:
predictions(a_list)

[5, 9, 9, 9, 7]

## Loss function

The loss (or cost) function is denoted by $C$. 

$$ C = \frac{1}{2 n} \sum_\mathbf{x} (\mathbf{a}^{(L)}(\mathbf{x}) - \mathbf{y}(\mathbf{x})) \cdot (\mathbf{a}^{(L)}(\mathbf{x}) - \mathbf{y}(\mathbf{x}))$$

When we get the $\mathbf{y}$ labels from the dataset they are, for example if there were four labels, of the form `[0, 5, 4, 9]`. In order to calculate the the loss properly we must put this into a one-hot-encoded form. The `[0, 5, 4, 9]` would then become a matrix looking like this

$$\begin{bmatrix} 
1 && 0 && 0 && 0 && 0 && 0 && 0 && 0 && 0 && 0 \\ 
0 && 0 && 0 && 0 && 0 && 1 && 0 && 0 && 0 && 0 \\
0 && 0 && 0 && 0 && 1 && 0 && 0 && 0 && 0 && 0 \\
0 && 0 && 0 && 0 && 0 && 0 && 0 && 0 && 0 && 1 \\
\end{bmatrix} $$

In [17]:
## Under construction ----

def loss(y_preds, y_targets):
    """Calculates the loss"""
    preds = y_preds[-1] # Pick the final layer's values.
    
    # Create a matrix where each row corresponds to an observation
    # and has ten columns. In every row only one value is 1 and 
    # the rest are zero.
    y_one_hot = np.zeros((preds.shape[0],10))
    for i, target in enumerate(y_targets):
        y_one_hot[i, target] = 1
    return np.matmul(y_one_hot, preds.T)

In [18]:
loss(a_list, y_train[:5])

array([[3.55797578e-01, 1.78827930e-02, 1.30770317e-02, 1.41190243e-03,
        6.59157218e-06],
       [3.04402644e-01, 1.38662592e-06, 4.39282459e-05, 1.02904414e-02,
        1.89234252e-03],
       [1.72990318e-03, 4.32818289e-08, 2.84885608e-06, 4.17324496e-06,
        1.60749015e-07],
       [3.11284123e-01, 1.63739972e-02, 1.40226780e-01, 3.36896009e-01,
        1.42743093e-01],
       [6.09214579e-03, 9.62343735e-01, 8.38753609e-01, 6.38263699e-01,
        2.48931732e-02]])

## Backpropagation

$$\delta_j^l = \frac{\partial C}{\partial z_j^l}$$



In [51]:
def backpropagation(x, y, weights, biases, activations):
    """Stuff..."""
    y = one_hot(y)
    z_list, a_list = feedforward(x, weights, biases, activations)
    DELTAs = []
    delC_delbs = []
    delC_delWs = []
    
    # Since we propagate backwards, we work with all matrices in reverse order
    reverse_list = np.flip(list(zip(z_list, a_list, activations)), axis=0)
    
    for i, (z, a, activation) in enumerate(reverse_list):
        if i == 0:
            DELTA = (a - z) * activation(z, deriv=True)
        else:
            DELTA = np.matmul(DELTA, weights[-i]) * activation(z, deriv=True)
        DELTAs.append(DELTA)
        
     
    for i, DELTA in enumerate(DELTAs):
        delC_delb = DELTA
        delC_delw = np.matmul(a_list[-i-1], DELTA.T)
        
        delC_delb = delC_delb.mean(axis=1)
        delC_delw = delC_delw.mean(axis=1)
        
        delC_delbs.append(delC_delb)
        delC_delWs.append(delC_delw)
    
    delC_delWs = np.flip(delC_delWs, axis=0)
    delC_delbs = np.flip(delC_delbs, axis=0)
    
    #return delC_delWs, delC_delbs
    return DELTAs


def one_hot(y):
    # Create a matrix where each row corresponds to an observation
    # and has ten columns. In every row only one value is 1 and 
    # the rest are zero.
    y_one_hot = np.zeros((len(y),10))
    for i, target in enumerate(y):
        y_one_hot[i, target] = 1
    return y_one_hot

X = x_train[:5]
y = y_train[:5]

DELTAs = backpropagation(X, y, weights, biases, [sigmoid, sigmoid, sigmoid])
#delC_delws, delC_delbs = backpropagation(X, y, weights, biases, [sigmoid, sigmoid, sigmoid])

In [52]:
DELTAs

[array([[-5.09035837e-03, -4.99628278e-03, -1.03151085e-01,
          2.90652407e-05, -1.16780173e-01, -4.46769919e-03,
         -4.87255638e-02, -7.11245876e-02,  4.42939479e-02,
         -8.53708206e-02],
        [ 1.83894081e-01, -5.86507530e-03,  1.19373035e-02,
          3.98561448e-04,  1.56989807e-02, -5.45109028e-03,
         -1.02170385e-01, -2.22351281e-02, -3.61679205e-02,
         -1.68829422e-04],
        [ 2.07131622e-01, -3.40956254e-03,  2.54788507e-01,
          5.10113164e-06,  1.18672680e-01, -2.31363784e-02,
         -6.06354669e-02, -4.83255686e-02,  1.08310716e-01,
         -7.25995796e-04],
        [-2.40769841e-02, -1.36935032e-03, -4.21952962e-02,
          2.11638466e-05,  1.65267470e-01, -8.63514040e-02,
          1.69588612e-02, -2.80295167e-02,  1.79006225e-01,
         -7.83484545e-04],
        [-5.78646151e-02, -1.98868313e-03,  6.70900695e-03,
          8.68499423e-05,  2.22414709e-02,  2.36990893e-01,
          2.54919431e-01, -4.25691700e-04,  4.154547

Now we define the loss function

In [81]:
def loss(output, labels):
    return ((output - labels)**2).sum(axis=1) / output.shape[1]

Now comes the big and scary backpropagation part. We must find $\frac{\partial Loss}{\partial W^{(l)}}$ and $\frac{\partial Loss}{\partial b^{(l)}}$ for l = 1, 2, 3.



In [82]:
labels = y_train[:batch_size]

tmp = np.zeros((batch_size, 10))

for i, d in enumerate(labels):
    tmp[i, d] = 1
    
labels = tmp

In [83]:
loss(output, labels)

array([1.99998005e-01, 1.99867436e-01, 1.99396618e-01, 1.99388007e-01,
       1.99506456e-01, 1.46568314e-01, 1.71471797e-01, 1.99966959e-01,
       1.44171031e-01, 1.47242352e-01, 1.98174972e-01, 1.66822791e-01,
       1.58642851e-01, 1.31150678e-06, 1.98073519e-01, 1.99877940e-01,
       1.51632337e-01, 1.83299589e-01, 2.10432617e-08, 1.99595395e-01,
       1.91602149e-01, 1.99200588e-01, 1.89762059e-01, 1.99939249e-01,
       1.57726825e-01, 1.46877428e-01, 1.93640505e-01, 1.99966266e-01,
       5.63214947e-02, 1.80340827e-01, 1.49772316e-01, 1.97400331e-01,
       4.40508189e-10, 1.99604068e-01, 1.96435548e-01, 1.26200057e-01,
       5.56476585e-14, 1.79196392e-01, 1.43070049e-01, 1.81084066e-02,
       1.65507017e-01, 1.48002526e-01, 4.08854518e-02, 1.74206957e-01,
       8.99279820e-02, 1.95010278e-01, 1.53848672e-01, 1.99701525e-01,
       1.99934981e-01, 1.99854615e-01, 1.85290629e-01, 1.99981499e-01,
       7.75641569e-02, 1.99295908e-01, 1.11112251e-01, 1.93942563e-01,
      